In [1]:
import pandas as pd   
import pandas_gbq as pgbq 
import os
import re
import sys
path_note = os.getcwd()
path_src = re.sub(r'(.)notebooks','',path_note)
path_src_resource = re.sub(r'(.)notebooks','\/v0',path_note)
path_src_conectores = re.sub(r'(.)notebooks','\/v0\/conectores',path_note)

In [2]:
print(path_src_resource.replace('/',''))

c:\Users\andre\Documents\github\my-money-family\admin\src\v0


In [3]:
import sys
sys.path.append(os.path.abspath(f'{path_src}/'))
sys.path.append(os.path.abspath(f'{path_src_resource}/'))
sys.path.append(os.path.abspath(f'{path_src_conectores}/'))

In [109]:
# from v0.resource.fin_api import consulta_gcp
# strsql = f"SELECT * FROM dev_domestico.custo_2024_excel  "

# dfc = consulta_gcp(strsql)

from v0.resource.myprocessanalytics_v0 import process
strsql = f"SELECT * FROM dev_domestico.custo_2024_excel "

dfc = process(strsql, flag_fake=False)

dfc = dfc[['tipo_custo','custo','valor_custo','dt_mes_base','dt_custo','tipo_custo_alt','classificacao_custo','area_custo']].drop_duplicates()

In [7]:
from v0.resource.fin_api import consulta_gcp
strsql = f"SELECT * FROM dev_domestico.recebido_2024_excel  "

dfr = consulta_gcp(strsql)

In [114]:
dfc[(pd.to_datetime(dfc['dt_mes_base'])=='2024-03-01') ]

,tipo_custo,custo,valor_custo,dt_mes_base,dt_custo,tipo_custo_alt,classificacao_custo,area_custo
720,sabesp,SABESP 3082129325,241.30,2024-03-01,2024-03-01,sabesp,fixo,medio
721,alimentacao,PIX QRS Tuna Pagame01/03,33.99,2024-03-01,2024-03-01,alimentacao,variado,medio_n1
722,gasolina,ELECTRONCENTRO AUTO,50.00,2024-03-01,2024-03-04,gasolina,variado,carro
723,alimentacao,PIX QRS CANTINA ERI03/03,2.00,2024-03-01,2024-03-04,alimentacao,variado,medio_n1
724,gasolina,ELECTRONCENTRO AUTO,100.00,2024-03-01,2024-03-04,gasolina,variado,carro
...,...,...,...,...,...,...,...,...
826,gasolina,ELECTRONPOSTO SERVC,50.00,2024-03-01,2024-03-25,gasolina,variado,carro
827,alimentacao,ELECTRONLUIZ ANTONI,47.00,2024-03-01,2024-03-25,alimentacao,variado,medio_n1
828,feira,ELECTRONPAG*Rosilen,13.00,2024-03-01,2024-03-25,feira,variado,medio_n1
829,app,PIX QRS MOBIAUTO ED26/03,99.00,2024-03-01,2024-03-26,app,variado,imprevisto


In [8]:
from v0.resource.fin_api import consulta_gcp
strsql = f"SELECT * FROM dev_domestico.saldo_2024_excel  "

dfs = consulta_gcp(strsql)

In [111]:
def tab_analytics():

    def agrup_saldo(dfs, dt):
        max_data_saldo = dfs['dt_recebido'][(dfs['saldo']!=1.00) & ((pd.to_datetime(dfs['dt_mes_base'])==dt)) ].max()
        valor_saldo_ = dfs['saldo'][(pd.to_datetime(dfs['dt_mes_base'])==dt) & (pd.to_datetime(dfs['dt_recebido'])==max_data_saldo.strftime("%Y-%m-%d"))].max()
        return valor_saldo_ 

    def func_recebido(dfr, mes):
        if mes<=9:
            if mes==4:
                value = dfr['valor_recebido'].loc[(pd.to_datetime(dfr['dt_mes_base'])==f'2024-0{mes}-01')].sum() 
            else:
                value = dfr['valor_recebido'].loc[(pd.to_datetime(dfr['dt_mes_base'])==f'2024-0{mes}-01')].sum()
        else:
            value = dfr['valor_recebido'].loc[(pd.to_datetime(dfr['dt_mes_base'])==f'2024-{mes}-01')].sum()
        return value

        
    def func_custo(dfc, mes):
        if mes<=9:
            if mes==4:
                value = dfc['valor_custo'].loc[(pd.to_datetime(dfc['dt_mes_base'])==f'2024-0{mes}-01')].sum()
            else:
                value = dfc['valor_custo'].loc[(pd.to_datetime(dfc['dt_mes_base'])==f'2024-0{mes}-01')].sum()
        else:
            value = dfc['valor_custo'].loc[(pd.to_datetime(dfc['dt_mes_base'])==f'2024-{mes}-01')].sum()
        return value

    def func_custo_fixo(dfc, mes):
        if mes<=9:
            if mes==4:
                value = dfc['valor_custo'].loc[(pd.to_datetime(dfc['dt_mes_base'])==f'2024-0{mes}-01')  & (dfc['classificacao_custo']=='fixo') ].sum()
            else:
                value = dfc['valor_custo'].loc[(pd.to_datetime(dfc['dt_mes_base'])==f'2024-0{mes}-01')  & (dfc['classificacao_custo']=='fixo')].sum()
        else:
            value = dfc['valor_custo'].loc[(pd.to_datetime(dfc['dt_mes_base'])==f'2024-{mes}-01')  & (dfc['classificacao_custo']=='fixo')].sum()
        return value

    def func_custo_var(dfc, mes):
        if mes<=9:
            if mes==4:
                value = dfc['valor_custo'].loc[(pd.to_datetime(dfc['dt_mes_base'])==f'2024-0{mes}-01') & (dfc['classificacao_custo']=='variado')].sum()
            else:
                value = dfc['valor_custo'].loc[(pd.to_datetime(dfc['dt_mes_base'])==f'2024-0{mes}-01') & (dfc['classificacao_custo']=='variado')].sum()
        else:
            value = dfc['valor_custo'].loc[(pd.to_datetime(dfc['dt_mes_base'])==f'2024-{mes}-01') & (dfc['classificacao_custo']=='variado')].sum()
        return value
            
    def func_saldo(dfs, mes):
        if mes==0:
            value = 9711.04
        elif mes<=9:
            value = agrup_saldo(dfs, f'2024-0{mes}-01')
            value = value -func_poupanca(dfr, mes-1)
        else:
            value = agrup_saldo(dfs, f'2024-{mes}-01')
            value = value -func_poupanca(dfr, mes-1)
        return value
    
    def func_poupanca(dfr, mes):
        if mes<=9:
            value = dfr['valor_recebido'].loc[(pd.to_datetime(dfr['dt_mes_base'])==f'2024-0{mes}-01') & (dfr['descricao']=='TBI 9293.22915-0/500')].sum()
        else:
            value = dfr['valor_recebido'].loc[(pd.to_datetime(dfr['dt_mes_base'])==f'2024-{mes}-01') & (dfr['descricao']=='TBI 9293.22915-0/500')].sum()
        return value
    
    def func_calc_fin(dfs, dfc, dfr, mes):
        value = ((func_saldo(dfs, mes-1)+func_recebido(dfr, mes))-func_custo(dfc, mes))

        return value
    
    def func_analise(dfs, dfc, dfr, mes):

        value=(0 if ( func_calc_fin(dfs, dfc, dfr, mes) < 0 and func_saldo(dfs, mes) < 0 ) else 1)
        if value==0:
            msg='Negativo'
        else:
            msg='Positivo'
        
        return msg

    # FAKE

    def func_recebido_fake(dfr, mes):
        if mes<=9:
            if mes==4:
                value = dfr['valor_recebido'].loc[(pd.to_datetime(dfr['dt_mes_base'])==f'2024-0{mes}-01')& (~dfr['descricao'].isin(['TBI 9293.22915-0/500','SISPAG PIX  INSTACARRO S']))].sum() 
            else:
                value = dfr['valor_recebido'].loc[(pd.to_datetime(dfr['dt_mes_base'])==f'2024-0{mes}-01') & (~dfr['descricao'].isin(['TBI 9293.22915-0/500','SISPAG PIX  INSTACARRO S']))].sum()
        else:
            value = dfr['valor_recebido'].loc[(pd.to_datetime(dfr['dt_mes_base'])==f'2024-{mes}-01') & (~dfr['descricao'].isin(['TBI 9293.22915-0/500','SISPAG PIX  INSTACARRO S']))].sum()
        return value
    

    
    def func_custo_fake(dfc, mes):
        if mes<=9:
            if mes==4:
                value = dfc['valor_custo'].loc[(pd.to_datetime(dfc['dt_mes_base'])==f'2024-0{mes}-01')  & (~dfc['tipo_custo'].isin(['poupanca'])) ].sum()
            else:
                value = dfc['valor_custo'].loc[(pd.to_datetime(dfc['dt_mes_base'])==f'2024-0{mes}-01')].sum()
        else:
            value = dfc['valor_custo'].loc[(pd.to_datetime(dfc['dt_mes_base'])==f'2024-{mes}-01')].sum()
        return value

    def func_saldo_inicial_0():
        
        return 9711.04

    def func_saldo_inicial_fake(dfr, dfc, mes):

        rec_value = func_recebido_fake(dfr, mes)
        cst_value = func_custo_fake(dfc, mes)

        value = (func_saldo_inicial_0()+rec_value) - cst_value            
        return value

    def func_saldo_dinamico_fake(saldo, dfr, dfc, mes):
        if mes==1:
            # rec_value = func_recebido_fake(dfr, mes)
            # cst_value = func_custo_fake(dfc, mes)

            value = saldo 
        else:
            rec_value = func_recebido_fake(dfr, mes)
            cst_value = func_custo_fake(dfc, mes)

            value = func_saldo_inicial_0()-(saldo+rec_value) - cst_value  
                          
        return value
    
    # resultado_202412=(msg_negativa if ( calc_financeiro_202412 < 0 and valor_saldo_202412 < 0 ) else msg)
    df_ = pd.DataFrame()
    df_ = pd.DataFrame({'analise':[ func_analise(dfs, dfc, dfr, s) for s in range(1,13) ],
    'valor_saldo_inicial':[ func_saldo(dfs, s-1) for s in range(1,13) ],
    'valor_recebido':[ func_recebido(dfr, s)  for s in range(1,13) ],

    'valor_poupanca':[ func_poupanca(dfr, s) for s in range(1,13) ],
    'valor_custo':[ func_custo(dfc, s) for s in range(1,13) ],
    'valor_custo_fixo':[ func_custo_fixo(dfc, s) for s in range(1,13) ],
    'valor_custo_var':[ func_custo_var(dfc, s) for s in range(1,13) ],
    

    'valor_rec_x_cust':[ func_recebido(dfr, s)-func_custo(dfc, s)-func_poupanca(dfr, s)  for s in range(1,13) ],
    'valor_saldo_final':[func_saldo(dfs, s)  for s in range(1,13) ],
    'analise_calculo_financeiro':[ func_calc_fin(dfs, dfc, dfr, s) for s in range(1,13) ],

    ## pega mes passado para calc saldo inicial
    'valor_saldo_ini_fake':[ func_saldo_inicial_0() if s==1 
                                                    else func_recebido_fake(dfr, s-1)-func_custo_fake(dfc, s-1) for s in range(1,13)  ],

    'valor_recebido_fake':[ func_recebido_fake(dfr, s) for s in range(1,13) ],
    'valor_custo_fake':[ func_custo_fake(dfc, s) for s in range(1,13) ],
    ## pega mes corrente para calc do saldo final do mes
    'valor_saldo_fake':[ func_saldo_inicial_fake(dfr, dfc, s)  if s==1 else  func_recebido_fake(dfr, s)-func_custo_fake(dfc, s)  for s in range(1,13) ],

    'dt_mes_base':[pd.Timestamp(f"2024-0{s}-01") if s<=9 else pd.Timestamp(f"2024-{s}-01")  for s in range(1,13) ]
    })
 

    return df_

PREVISÃO DO MES DE JANEIRO 2024 - SALDO DO 2023

In [264]:
df_ = tab_analytics()


In [122]:
df_[['analise', 'valor_saldo_inicial', 'valor_recebido', 'valor_poupanca', 'valor_custo','valor_custo_fixo','valor_custo_var', 'valor_rec_x_cust', 'valor_saldo_final','analise_calculo_financeiro', 'dt_mes_base']]

,analise,valor_saldo_inicial,valor_recebido,valor_poupanca,valor_custo,valor_custo_fixo,valor_custo_var,valor_rec_x_cust,valor_saldo_final,analise_calculo_financeiro,dt_mes_base
0,Positivo,9711.04,6891.38,0.00,13876.84,8241.91,5634.93,-6985.46,2725.58,2725.58,2024-01-01
1,Positivo,2725.58,8034.93,0.00,9558.33,6240.28,3318.05,-1523.40,1202.18,1202.18,2024-02-01
2,Negativo,1202.18,10687.92,0.00,14279.14,8040.15,6238.99,-3591.22,-2391.94,-2389.04,2024-03-01
3,Negativo,-2391.94,30976.38,5000.00,31509.21,26182.81,5326.40,-5532.83,-2948.67,-2924.77,2024-04-01
4,Positivo,-2948.67,14939.64,7076.20,11601.21,8592.39,3008.82,-3737.77,-4610.24,389.76,2024-05-01
5,Negativo,-4610.24,12810.00,0.00,11474.39,9092.49,2381.90,1335.61,-5350.83,-3274.63,2024-06-01
6,Positivo,-5350.83,10662.49,1569.23,10309.67,7316.41,2993.26,-1216.41,2078.19,-4998.01,2024-07-01
7,Negativo,2078.19,7296.08,0.00,11641.52,7975.93,3665.59,-4345.44,-3836.48,-2267.25,2024-08-01
8,Negativo,-3836.48,10697.53,2751.51,10979.62,7427.40,3552.22,-3033.60,-2557.34,-4118.57,2024-09-01
9,Negativo,-2557.34,10314.81,1810.30,11434.69,8191.00,3243.69,-2930.18,-6428.73,-3677.22,2024-10-01


In [169]:
df_["valor_recebido"] = df_["valor_recebido"].astype(str)
df_["valor_recebido"] = df_["valor_recebido"].apply(lambda v: str(v.replace(".", ",")))

df_["valor_custo"] = df_["valor_custo"].astype(str)
df_["valor_custo"] = df_["valor_custo"].apply(lambda v: str(v.replace(".", ",")))

df_["valor_custo_fixo"] = df_["valor_custo_fixo"].astype(str)
df_["valor_custo_fixo"] = df_["valor_custo_fixo"].apply(lambda v: str(v.replace(".", ",")))

df_["valor_custo_var"] = df_["valor_custo_var"].astype(str)
df_["valor_custo_var"] = df_["valor_custo_var"].apply(lambda v: str(v.replace(".", ",")))

In [170]:
import csv
df_[['analise', 'valor_saldo_inicial', 'valor_recebido', 'valor_poupanca', 'valor_custo','valor_custo_fixo','valor_custo_var', 'valor_rec_x_cust', 'valor_saldo_final','analise_calculo_financeiro', 'dt_mes_base']].to_csv('C:\\Users\\andre\\Documents\\github\\my-money-family\\admin\\src\\v0\\data\plano_acao_analytics.csv', sep=';', mode='w', quoting=csv.QUOTE_ALL,  encoding='utf-8', doublequote=True, index=False)

In [265]:
def func_calc_saldo_dinamico(df_):
    list_df = df_[['valor_saldo_ini_fake','valor_recebido_fake','valor_custo_fake','valor_saldo_fake']].values.tolist()
    list_saldo_ini = []
    list_saldo_fin = []
    #for idx, row in df.iterrows():
    for idx, rw in enumerate(list_df):
        # print(idx)
        # print(rw)
        # print(list_df[idx-1][3])
        if idx==0:
            list_saldo_ini.append( list_df[idx][0] ) 
            list_saldo_fin.append( list_df[idx][3] )
        else:
            list_saldo_ini.append( list_df[idx-1][0]+list_df[idx][0] )
            list_saldo_fin.append( list_df[idx-1][3]+list_df[idx][1]-list_df[idx][2] )  #+int(list_df[idx][2])*(-1)
        # df_["valor_saldo_ini_fake_y"] = df_.apply(lambda x: x['valor_saldo_ini_fake_x']+x['valor_recebido_fake']-x['valor_custo_fake'], axis=1)
    # df_['valor_saldo_ini_fake'] = df_.apply(lambda x: , axis=1)

    return list_saldo_fin

In [255]:
func_calc_saldo_dinamico(df_)

[9711.04,
 2725.580000000001,
 -8508.86,
 -5114.619999999999,
 -9214.719999999998,
 -9361.269999999999,
 -2402.159999999999,
 119.19999999999891,
 -5561.849999999999,
 -7379.039999999996,
 -5963.779999999999,
 -4302.020000000002]

In [266]:
for n in range(1,12):
    #df_['valor_saldo_ini_fake']=func_calc_saldo_dinamico(df_)
    df_['valor_saldo_fake']=func_calc_saldo_dinamico(df_)

    df_[['valor_saldo_ini_fake','valor_recebido_fake','valor_custo_fake','valor_saldo_fake','dt_mes_base']]

In [267]:
df_[['valor_saldo_ini_fake','valor_recebido_fake','valor_custo_fake','valor_saldo_fake','dt_mes_base']]

,valor_saldo_ini_fake,valor_recebido_fake,valor_custo_fake,valor_saldo_fake,dt_mes_base
0,9711.04,6891.38,13876.84,2725.58,2024-01-01
1,-6985.46,8034.93,9558.33,1202.18,2024-02-01
2,-1523.40,10687.92,14279.14,-2389.04,2024-03-01
3,-3591.22,7885.71,13509.21,-8012.54,2024-04-01
4,-5623.50,7863.44,11601.21,-11750.31,2024-05-01
5,-3737.77,12810.00,11474.39,-10414.70,2024-06-01
6,1335.61,9093.26,10309.67,-11631.11,2024-07-01
7,-1216.41,7296.08,11641.52,-15976.55,2024-08-01
8,-4345.44,7946.02,10979.62,-19010.15,2024-09-01
9,-3033.60,8504.51,11434.69,-21940.33,2024-10-01


In [268]:
def func_calc_saldo_dinamico(df_):
    list_df = df_[['valor_saldo_ini_fake','valor_recebido_fake','valor_custo_fake','valor_saldo_fake']].values.tolist()
    list_saldo_ini = []

    #for idx, row in df.iterrows():
    for idx, rw in enumerate(list_df):
        # print(idx)
        # print(rw)
        # print(list_df[idx-1][3])
        if idx==0:
            list_saldo_ini.append( list_df[idx][0] ) 

        else:
            list_saldo_ini.append( list_df[idx-1][3] )
        #+int(list_df[idx][2])*(-1)
        # df_["valor_saldo_ini_fake_y"] = df_.apply(lambda x: x['valor_saldo_ini_fake_x']+x['valor_recebido_fake']-x['valor_custo_fake'], axis=1)
        # df_['valor_saldo_ini_fake'] = df_.apply(lambda x: , axis=1)

    return list_saldo_ini

In [269]:
func_calc_saldo_dinamico(df_)

[9711.04,
 2725.5800000000017,
 1202.180000000002,
 -2389.0399999999972,
 -8012.539999999996,
 -11750.309999999996,
 -10414.699999999995,
 -11631.109999999997,
 -15976.549999999996,
 -19010.149999999994,
 -21940.329999999994,
 -23312.17]

In [270]:
df_['valor_saldo_ini_fake']=func_calc_saldo_dinamico(df_)

In [271]:
df_[['valor_saldo_ini_fake','valor_recebido_fake','valor_custo_fake','valor_saldo_fake','dt_mes_base']]

,valor_saldo_ini_fake,valor_recebido_fake,valor_custo_fake,valor_saldo_fake,dt_mes_base
0,9711.04,6891.38,13876.84,2725.58,2024-01-01
1,2725.58,8034.93,9558.33,1202.18,2024-02-01
2,1202.18,10687.92,14279.14,-2389.04,2024-03-01
3,-2389.04,7885.71,13509.21,-8012.54,2024-04-01
4,-8012.54,7863.44,11601.21,-11750.31,2024-05-01
5,-11750.31,12810.00,11474.39,-10414.70,2024-06-01
6,-10414.70,9093.26,10309.67,-11631.11,2024-07-01
7,-11631.11,7296.08,11641.52,-15976.55,2024-08-01
8,-15976.55,7946.02,10979.62,-19010.15,2024-09-01
9,-19010.15,8504.51,11434.69,-21940.33,2024-10-01


In [272]:
df_["valor_saldo_ini_fake"] = df_["valor_saldo_ini_fake"].astype(str)
df_["valor_saldo_ini_fake"] = df_["valor_saldo_ini_fake"].apply(lambda v: str(v.replace(".", ",")))

df_["valor_recebido_fake"] = df_["valor_recebido_fake"].astype(str)
df_["valor_recebido_fake"] = df_["valor_recebido_fake"].apply(lambda v: str(v.replace(".", ",")))

df_["valor_custo_fake"] = df_["valor_custo_fake"].astype(str)
df_["valor_custo_fake"] = df_["valor_custo_fake"].apply(lambda v: str(v.replace(".", ",")))

df_["valor_saldo_fake"] = df_["valor_saldo_fake"].astype(str)
df_["valor_saldo_fake"] = df_["valor_saldo_fake"].apply(lambda v: str(v.replace(".", ",")))



In [273]:
df_[['valor_saldo_ini_fake','valor_recebido_fake','valor_custo_fake','valor_saldo_fake','dt_mes_base']]

,valor_saldo_ini_fake,valor_recebido_fake,valor_custo_fake,valor_saldo_fake,dt_mes_base
0,"9711,04","6891,38","13876,84","2725,5800000000017",2024-01-01
1,"2725,5800000000017","8034,93","9558,33","1202,180000000002",2024-02-01
2,"1202,180000000002","10687,92","14279,14","-2389,0399999999972",2024-03-01
3,"-2389,0399999999972","7885,71","13509,21","-8012,539999999996",2024-04-01
4,"-8012,539999999996","7863,44","11601,21","-11750,309999999996",2024-05-01
5,"-11750,309999999996","12810,0","11474,39","-10414,699999999995",2024-06-01
6,"-10414,699999999995","9093,26","10309,670000000002","-11631,109999999997",2024-07-01
7,"-11631,109999999997","7296,080000000001","11641,519999999999","-15976,549999999996",2024-08-01
8,"-15976,549999999996","7946,02","10979,619999999999","-19010,149999999994",2024-09-01
9,"-19010,149999999994","8504,51","11434,69","-21940,329999999994",2024-10-01


In [274]:
import csv
df_[['valor_saldo_ini_fake','valor_recebido_fake','valor_custo_fake','valor_saldo_fake','dt_mes_base']].to_csv('C:\\Users\\andre\\Documents\\github\\my-money-family\\admin\\src\\v0\\data\plano_acao.csv', sep=';', mode='w', quoting=csv.QUOTE_ALL,  encoding='utf-8', doublequote=True, index=False)

: 

In [4]:
from googleapiclient.discovery import build
import google.auth
from conecta.credencial_gcp import cred

import os

ModuleNotFoundError: No module named 'googleapiclient'

In [ ]:

sheetId = "C:\\Users\\andre\\Documents\\github\\my-money-family\\admin\\src\\v0\\data\"

# insert no sheet
df_sheet = df_ #[['instrument', 'categ', 'type', 'amount', 'close_time','hh_br', 'week_br', 'periodo_br', 'open_price_x', 'close_price_x', 'lucro_real', 'invest', 'fund_bruto',  'recupera_invest', 'tempo_oc_aberto', 'tempo_operando','tempo_operando_x']].copy()

#tratar dataframe
# df_sheet['close_time']=df_sheet['close_time'].astype(str)

# df_sheet = df_sheet.fillna(0)

df_sheet.

# myBody = {u'range': self.range_name_upd_xtb, u'values': df_sheet.T.reset_index().T.values.tolist(), u'majorDimension': u'ROWS'}

# credencial = cred()
# spreadsheet_service = build('sheets', 'v4', credentials=credencial[0])


# response_date = spreadsheet_service.spreadsheets().values().update(
#     spreadsheetId=sheetId,
#     valueInputOption='RAW',
#     range=self.range_name_upd_xtb,
#     body=myBody
# ).execute()

In [100]:
dfc[(dfc['tipo_custo'].isin(['poupanca']))][['valor_custo']].sum()

valor_custo    18000.0
dtype: float64

In [99]:
dfr[(dfr['descricao']=='TBI 9293.22915-0/500')][['valor_recebido']].sum()

valor_recebido    18207.24
dtype: float64

In [90]:
# import numpy as np
# # Calcular a média aritmétia, mediana, variância e desvio-padrão
# arr = np.array(df[['lucro_real']])
# # mediana = np.median(df)
# # media = np.mean(df)
# # desv_pad = np.std(df)
# # variancia = np.var(df)
# mediana = np.median(df[['lucro_real']])
# media = np.mean(df[['lucro_real']])
# desv_pad = np.std(df[['lucro_real']])
# variancia = np.var(df[['lucro_real']])

In [91]:
# from myextract import my_extract_excel_deb, my_type_modal_finance #my_extract_excel_cred
# my_fin = my_extract_excel_deb

# df = my_fin(path_src, f'custo_2023_08.xls')


In [92]:


    # import pandas as pd
    # import datetime as dt
    # from datetime import timedelta

    # valor_saldo_202400 = 9711.04 # marco zero


    # valor_saldo_202401=agrup_saldo(dfs, '2024-01-01')
    # valor_saldo_202402=agrup_saldo(dfs, '2024-02-01')
    # valor_saldo_202403=agrup_saldo(dfs, '2024-03-01')
    # valor_saldo_202404=agrup_saldo(dfs, '2024-04-01')
    # valor_saldo_202405=agrup_saldo(dfs, '2024-05-01')
    # valor_saldo_202406=agrup_saldo(dfs, '2024-06-01')
    # valor_saldo_202407=agrup_saldo(dfs, '2024-07-01')
    # valor_saldo_202408=agrup_saldo(dfs, '2024-08-01')
    # valor_saldo_202409=agrup_saldo(dfs, '2024-09-01')
    # valor_saldo_202410=agrup_saldo(dfs, '2024-10-01')
    # valor_saldo_202411=agrup_saldo(dfs, '2024-11-01')
    # valor_saldo_202412=agrup_saldo(dfs, '2024-12-01')


    # # data_open='2023-08-31'
    # # data_close='2023-09-07'
    # # total_valor_mercado = df['valor_custo'].loc[df['tipo_custo']=='mercado'].max()
    # valor_recebido_202401=dfr['valor_recebido'].loc[(pd.to_datetime(dfr['dt_mes_base'])=='2024-01-01')].sum()
    # valor_recebido_202402=dfr['valor_recebido'].loc[(pd.to_datetime(dfr['dt_mes_base'])=='2024-02-01')].sum()
    # valor_recebido_202403=dfr['valor_recebido'].loc[(pd.to_datetime(dfr['dt_mes_base'])=='2024-03-01')].sum()
    # valor_recebido_202404=dfr['valor_recebido'].loc[(pd.to_datetime(dfr['dt_mes_base'])=='2024-04-01')].sum()
    # valor_recebido_202405=dfr['valor_recebido'].loc[(pd.to_datetime(dfr['dt_mes_base'])=='2024-05-01')].sum()
    # valor_recebido_202406=dfr['valor_recebido'].loc[(pd.to_datetime(dfr['dt_mes_base'])=='2024-06-01')].sum()
    # #fake
    # valor_recebido_202407=dfr['valor_recebido'].loc[(pd.to_datetime(dfr['dt_mes_base'])=='2024-07-01')].sum()
    # valor_recebido_202408=dfr['valor_recebido'].loc[(pd.to_datetime(dfr['dt_mes_base'])=='2024-08-01')].sum()
    # valor_recebido_202409=dfr['valor_recebido'].loc[(pd.to_datetime(dfr['dt_mes_base'])=='2024-09-01')].sum()
    # valor_recebido_202410=dfr['valor_recebido'].loc[(pd.to_datetime(dfr['dt_mes_base'])=='2024-10-01')].sum()
    # valor_recebido_202411=dfr['valor_recebido'].loc[(pd.to_datetime(dfr['dt_mes_base'])=='2024-11-01')].sum()
    # valor_recebido_202412=dfr['valor_recebido'].loc[(pd.to_datetime(dfr['dt_mes_base'])=='2024-12-01')].sum()


    # valor_custo_202401=dfc['valor_custo'].loc[(pd.to_datetime(dfc['dt_mes_base'])=='2024-01-01')].sum()
    # valor_custo_202402=dfc['valor_custo'].loc[(pd.to_datetime(dfc['dt_mes_base'])=='2024-02-01')].sum()
    # valor_custo_202403=dfc['valor_custo'].loc[(pd.to_datetime(dfc['dt_mes_base'])=='2024-03-01') ].sum()
    # valor_custo_202404=dfc['valor_custo'].loc[(pd.to_datetime(dfc['dt_mes_base'])=='2024-04-01') & (~dfc['tipo_custo'].isin(['poupanca']))].sum()
    # valor_custo_202405=dfc['valor_custo'].loc[(pd.to_datetime(dfc['dt_mes_base'])=='2024-05-01')].sum()
    # valor_custo_202406=dfc['valor_custo'].loc[(pd.to_datetime(dfc['dt_mes_base'])=='2024-06-01')].sum()
    # #fake
    # valor_custo_202407=dfc['valor_custo'].loc[(pd.to_datetime(dfc['dt_mes_base'])=='2024-07-01')].sum()
    # valor_custo_202408=dfc['valor_custo'].loc[(pd.to_datetime(dfc['dt_mes_base'])=='2024-08-01')].sum()
    # valor_custo_202409=dfc['valor_custo'].loc[(pd.to_datetime(dfc['dt_mes_base'])=='2024-09-01')].sum()
    # valor_custo_202410=dfc['valor_custo'].loc[(pd.to_datetime(dfc['dt_mes_base'])=='2024-10-01')].sum()
    # valor_custo_202411=dfc['valor_custo'].loc[(pd.to_datetime(dfc['dt_mes_base'])=='2024-11-01')].sum()
    # valor_custo_202412=dfc['valor_custo'].loc[(pd.to_datetime(dfc['dt_mes_base'])=='2024-12-01')].sum()

    # msg_negativa='Retorno negativo'
    # msg='Entrada positiva obteve um retorno positivo'
    # calc_financeiro_202401 = ((valor_saldo_202400+valor_recebido_202401)-valor_custo_202401) #-valor_saldo_202401
    # resultado_202401=(msg_negativa if (calc_financeiro_202401 < 0 and valor_saldo_202401 < 0 ) else msg)

    # msg='Entrada/Saldo Inicial positiva obteve um retorno positivo'
    # calc_financeiro_202402 = ((valor_saldo_202401+valor_recebido_202402)-valor_custo_202402) #-valor_saldo_202402
    # resultado_202402=(msg_negativa if ( calc_financeiro_202402 < 0 and valor_saldo_202402 < 0 ) else msg)

    # calc_financeiro_202403 = ((valor_saldo_202402+valor_recebido_202403)-valor_custo_202403) #-valor_saldo_202403
    # resultado_202403=(msg_negativa if ( calc_financeiro_202403 < 0 and valor_saldo_202403 < 0 ) else msg)

    # msg='Entrada extra e inicio com saldo negativo obteve um retorno positivo'
    # saldo_atual_reserva = 18090.67	# 18000 - 5000 02/04 | 13000 - 1000 03/05 | 12076.20 - 6076.20 23/05
    # calc_financeiro_202404 = ((valor_saldo_202403+valor_recebido_202404)-valor_custo_202404)#-saldo_atual_reserva
    # resultado_202404=(msg_negativa if  ( calc_financeiro_202404 < 0 and valor_saldo_202404 < 0 ) else msg)

    # msg='Entrada extra e inicio com saldo negativo obteve um retorno positivo'
    # #saldo_atual_reserva = 6000 # 12076.20 - 6076.20 23/05
    # calc_financeiro_202405 = ((valor_saldo_202404+valor_recebido_202405)-valor_custo_202405) #-valor_saldo_202405
    # resultado_202405=(msg_negativa if  ( calc_financeiro_202405 < 0 and valor_saldo_202405 < 0 ) else msg)

    # msg='Entrada 13 / Inicio com saldo negativo mas obteve um retorno positivo'
    # saldo_atual_reserva = 0 # 12076.20 - 6076.20 23/05
    # calc_financeiro_202406 = ((valor_saldo_202405+valor_recebido_202406)-valor_custo_202406)+saldo_atual_reserva
    # resultado_202406=(msg_negativa if  ( calc_financeiro_202406 < 0 and valor_saldo_202406 < 0 ) else msg)

    # msg='Entrada extra e inicio com saldo negativo obteve um retorno positivo'
    # saldo_atual_reserva = 0 # 12076.20 - 6076.20 23/05 | 6069,23 - 1569.23 30/07 | 4500
    # calc_financeiro_202407 = ((valor_saldo_202406+valor_recebido_202407)-valor_custo_202407)+saldo_atual_reserva
    # resultado_202407=(msg_negativa if  ( calc_financeiro_202407 < 0 and valor_saldo_202407 < 0 ) else msg)

    # msg='Entrada extra / Inicio com saldo positivo obteve um retorno positivo'
    # saldo_atual_reserva = 0 #4500+4100 # 12076.20 - 6076.20 23/05 | 6069,23 - 1569.23 30/07 | 4500
    # calc_financeiro_202408 = ((valor_saldo_202407+valor_recebido_202408)-valor_custo_202408)+saldo_atual_reserva
    # resultado_202408=(msg if ( calc_financeiro_202408 < 0 and valor_saldo_202408 < 0 ) else msg)

    # msg='Inicio com saldo negativo obteve um retorno negativo'
    # saldo_atual_reserva = 0 #4500+4100 # 12076.20 - 6076.20 23/05 | 6069,23 - 1569.23 30/07 | 4500
    # calc_financeiro_202409 = ((valor_saldo_202408+valor_recebido_202409)-valor_custo_202409)+saldo_atual_reserva
    # resultado_202409=(msg if ( calc_financeiro_202409 < 0 and valor_saldo_202409 < 0 ) else msg)

    # msg='Entrada extra / Inicio com saldo negativo obteve um retorno negativo'
    # saldo_atual_reserva = 0 #4500+4100 # 12076.20 - 6076.20 23/05 | 6069,23 - 1569.23 30/07 | 4500
    # calc_financeiro_202410 = ((valor_saldo_202409+valor_recebido_202410)-valor_custo_202410)+saldo_atual_reserva
    # resultado_202410=(msg if ( calc_financeiro_202410 < 0 and valor_saldo_202410 < 0 ) else msg)

    # msg='Inicio com saldo negativo obteve um retorno negativo'
    # saldo_atual_reserva = 0 #4500+4100 # 12076.20 - 6076.20 23/05 | 6069,23 - 1569.23 30/07 | 4500
    # calc_financeiro_202411 = ((valor_saldo_202410+valor_recebido_202411)-valor_custo_202411)+saldo_atual_reserva
    # resultado_202411=(msg_negativa if ( calc_financeiro_202411 < 0 and valor_saldo_202411 < 0 ) else msg) 

    # msg='Inicio com saldo negativo obteve um retorno negativo'
    # saldo_atual_reserva = 0 #4500+4100 # 12076.20 - 6076.20 23/05 | 6069,23 - 1569.23 30/07 | 4500
    # calc_financeiro_202412 = ((valor_saldo_202411+valor_recebido_202412)-valor_custo_202412)+saldo_atual_reserva
    # resultado_202412=(msg_negativa if ( calc_financeiro_202412 < 0 and valor_saldo_202412 < 0 ) else msg) 

    # # print(calc_financeiro_202410)  
    # # print(valor_saldo_202410)    
    # #groupby(['dt_mes_base']).agg({"valor_recebido":sum}).sort_values("valor_recebido", ascending=False)
    # # total_valor_alimentacao = df['valor_custo'].loc[(df['tipo_custo']=='alimentacao') & ((pd.to_datetime(df['dt_mes_base'])=='2023-11-01'))].sum()
    # # total_valor_padaria = df['valor_custo'].loc[(df['tipo_custo']=='padaria') & ((pd.to_datetime(df['dt_mes_base'])=='2023-09-01'))].sum()
    # # df_grp_f = df[df['tipo_custo'].str.contains(r'feira-\w+|feira')==True].groupby('dt_mes_base')['valor_custo'].sum()
    # # total_valor_feira = df_grp_f.median()
    # #df.loc[(df['tipo_custo']=='padaria') & ((pd.to_datetime(df['dt_mes_base'])=='2023-09-01'))]
    # #dia = df_analytics["tempo_operando"].max()
    
    # # df_ = pd.DataFrame({'analise':[resultado_202401, resultado_202402, resultado_202403, resultado_202404, resultado_202405, resultado_202406, resultado_202407, resultado_202408, resultado_202409, resultado_202410, resultado_202411, resultado_202412],	
    # #                     'valor_saldo_inicial':[valor_saldo_202400	, valor_saldo_202401, valor_saldo_202402, valor_saldo_202403, valor_saldo_202404, valor_saldo_202405, valor_saldo_202406, valor_saldo_202407, valor_saldo_202408, valor_saldo_202409, valor_saldo_202410, valor_saldo_202411],
    # #                     'valor_recebido':[valor_recebido_202401, valor_recebido_202402, valor_recebido_202403, valor_recebido_202404, valor_recebido_202405, valor_recebido_202406, valor_recebido_202407, valor_recebido_202408, valor_recebido_202409, valor_recebido_202410, valor_recebido_202411, valor_recebido_202412],
    # #                     'valor_custo':[valor_custo_202401, valor_custo_202402, valor_custo_202403,valor_custo_202404,valor_custo_202405, valor_custo_202406, valor_custo_202407, valor_custo_202408, valor_custo_202409, valor_custo_202410, valor_custo_202411, valor_custo_202412],
    # #                     'valor_rec_x_cust':[valor_recebido_202401-valor_custo_202401, valor_recebido_202402-valor_custo_202402, valor_recebido_202403-valor_custo_202403,valor_recebido_202404-valor_custo_202404,valor_recebido_202405-valor_custo_202405, valor_recebido_202406-valor_custo_202406, valor_recebido_202407-valor_custo_202407, valor_recebido_202408-valor_custo_202408, valor_recebido_202409-valor_custo_202409, valor_recebido_202410-valor_custo_202410, valor_recebido_202411-valor_custo_202411,  valor_recebido_202412-valor_custo_202412],
    # #                     'valor_saldo_final':[valor_saldo_202401, valor_saldo_202402, valor_saldo_202403, valor_saldo_202404, valor_saldo_202405, valor_saldo_202406, valor_saldo_202407, valor_saldo_202408, valor_saldo_202409, valor_saldo_202410, valor_saldo_202411, valor_saldo_202412],
    # #                     'analise_calculo_financeiro':[calc_financeiro_202401, calc_financeiro_202402, calc_financeiro_202403, calc_financeiro_202404, calc_financeiro_202405, calc_financeiro_202406, calc_financeiro_202407, calc_financeiro_202408, calc_financeiro_202409, calc_financeiro_202410, calc_financeiro_202411, calc_financeiro_202412],
    # #                     'dt_mes_base':[pd.Timestamp("2024-01-01"), pd.Timestamp("2024-02-01"), pd.Timestamp("2024-03-01"), pd.Timestamp("2024-04-01"), pd.Timestamp("2024-05-01"), pd.Timestamp("2024-06-01"), pd.Timestamp("2024-07-01"), pd.Timestamp("2024-08-01"), pd.Timestamp("2024-09-01"), pd.Timestamp("2024-10-01"), pd.Timestamp("2024-11-01"), pd.Timestamp("2024-12-01")],
    # #                     }
    
    # # )

In [93]:
# df = df.loc[df['valor_alt'] > 0].copy()
# df = df[df['descricao'].str.contains(r'SALDO[^\\b]+\w')==False].copy()
# df = df[df['descricao'].str.contains(r'RES APLIC[^\\b]+\w')==False].copy()
# #df = df[df['descricao'].str.contains(r'REND PAGO[^\\b]+\w')==False].copy()

# my_type_modal_fin = my_type_modal_finance

# my_type_modal_fin(df_x, path_src, f'custo_2023_08.xls', 'pulling')
# print('\n')
# my_type_modal_fin(df_x, path_src, f'custo_2023_08.xls', 'pushout')
# print('\n')
# my_type_modal_fin(df_x, path_src, f'custo_2023_08.xls', 'stop')